In [13]:
from __future__ import division

import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Cropping2D

from keras import backend as K

import keras
import h5py

from keras.layers.normalization import BatchNormalization
import importlib

from keras.optimizers import Nadam
from keras.callbacks import History
import pandas as pd
from keras.backend import binary_crossentropy

import datetime
import os

import random
import threading
from tqdm import tqdm

from keras.models import model_from_json

In [8]:
def set_keras_backend(backend):
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend
        
def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)

def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    print(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

In [16]:
set_keras_backend("theano")
K.set_image_dim_ordering('th')
#jaccard_similarity([1.5,1.0], [1.0,0.9])

test_ids = ['6110_3_1','6120_2_2','6140_3_1','6110_1_2','6110_4_0','6120_2_0','6120_2_0']
data = pd.read_csv('../data/train_wkt_v4.csv')
result = pd.read_csv('../data/joined.csv')

In [15]:
for image_id in tqdm(test_ids):
    y_true = data[data['ImageId']==image_id]
    print(y_true)

100%|██████████| 7/7 [00:00<00:00, 61.28it/s]

      ImageId  ClassType                                    MultipolygonWKT
150  6110_3_1          1  MULTIPOLYGON (((0.0040671780447186 -0.00889538...
151  6110_3_1          2  MULTIPOLYGON (((0.000434018999609 -0.009035395...
152  6110_3_1          3  MULTIPOLYGON (((0.0050952230484001 -0.00222609...
153  6110_3_1          4  MULTIPOLYGON (((0.008656378941500001 -0.008587...
154  6110_3_1          5  MULTIPOLYGON (((0.0091183991668912 -0.00903939...
155  6110_3_1          6  MULTIPOLYGON (((0.0091664012682297 -0.00891339...
156  6110_3_1          7                                 MULTIPOLYGON EMPTY
157  6110_3_1          8  MULTIPOLYGON (((0.0042681868440771 -0.00358915...
158  6110_3_1          9  MULTIPOLYGON (((0.0018630815582277 -0.00510022...
159  6110_3_1         10  MULTIPOLYGON (((0.0010150444345697 -0.00718431...
     ImageId  ClassType                                    MultipolygonWKT
10  6120_2_2          1  MULTIPOLYGON (((0.000439 -0.009039999999999999...
11  6120_2_2  